In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import numpy as np

In [3]:
train = pd.read_csv('./data/dacon_g/train_668.csv', index_col='id')
test = pd.read_csv('./data/dacon_g/test_g+i.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col=0)

In [ ]:
# color 전처리 파일 load
train_color = pd.read_csv('./data/dacon_g/train_colorindex.csv', index_col='id')
test_color = pd.read_csv('./data/dacon_g/test_colorindex.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col='id')

In [ ]:
# g+i 전처리 파일 load
train_color = pd.read_csv('./data/dacon_g/train_g+i.csv', index_col='id')
test_color = pd.read_csv('./data/dacon_g/test_g+i.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col='id')

In [19]:
# train.head()
test.columns

Index(['Unnamed: 0', 'fiberID', 'psfMag_g', 'psfMag_i', 'fiberMag_g',
       'fiberMag_i', 'petroMag_g', 'petroMag_i', 'modelMag_g', 'modelMag_i',
       'fiber_u-g', 'fiber_g-r', 'fiber_r-i', 'fiber_i-z', 'psf_u-g',
       'psf_g-r', 'psf_r-i', 'psf_i-z', 'petro_u-g', 'petro_g-r', 'petro_r-i',
       'petro_i-z', 'model_u-g', 'model_g-r', 'model_r-i', 'model_i-z'],
      dtype='object')

In [5]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [ ]:
# color 전처리
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train_color['type_num'] = train_color['type'].apply(lambda x: to_number(x, column_number))

In [20]:
train_x = train.drop(columns=["Unnamed: 0",'type', 'type_num','petro_u-g'], axis=1)
train_y = train['type_num']
test_x = test.drop(columns=["Unnamed: 0",'petro_u-g'], axis=1)

In [21]:
train_x


,fiberID,psfMag_g,psfMag_i,fiberMag_g,fiberMag_i,petroMag_g,petroMag_i,modelMag_g,modelMag_i,fiber_u-g,...,psf_g-r,psf_r-i,psf_i-z,petro_g-r,petro_r-i,petro_i-z,model_u-g,model_g-r,model_r-i,model_i-z
id,,,,,,,,,,,,,,,,,,,,,
0,601,21.431953,21.176553,21.644453,21.387653,21.431636,21.145409,21.465534,21.020605,0.936856,...,0.117805,0.137595,0.005109,-0.046676,0.332902,0.722964,1.283708,0.101347,0.343582,-0.126735
1,788,20.708104,20.703420,21.029773,20.937731,20.778968,20.639812,20.758527,20.693389,0.839024,...,0.029254,-0.024570,0.230191,-0.110737,0.249893,-0.006847,0.734428,0.004602,0.060537,0.181074
2,427,16.727898,16.694640,17.033098,17.095999,16.738784,16.744210,16.697434,16.660177,1.138792,...,0.048221,-0.014963,0.052852,0.049910,-0.055336,-0.063796,1.120628,0.056186,-0.018929,-0.028751
3,864,20.040371,19.843840,20.317165,20.073852,19.993727,19.750917,20.001699,19.758113,0.721865,...,0.113462,0.083069,0.380569,0.008197,0.234614,0.295800,0.769012,0.111901,0.131685,0.205258
4,612,23.058767,19.504961,23.629122,19.861718,22.426929,19.610012,23.147993,19.487330,2.071510,...,1.587361,1.966445,1.115864,0.753377,2.063539,1.233871,1.729059,1.672651,1.988012,1.111675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,568,20.437920,20.020974,20.726709,20.323418,20.536338,20.102815,20.326879,20.061940,0.221810,...,0.387163,0.029782,0.061328,0.409777,0.023746,0.306092,0.456416,0.312833,-0.047895,0.199428
199987,362,22.807243,20.187531,22.818723,19.961857,22.250489,18.985791,21.952440,18.990039,3.858948,...,1.737451,0.882262,0.591933,2.158797,1.105902,0.182437,4.969463,2.176119,0.786282,0.461733
199988,146,21.902709,18.457535,22.287946,18.810560,22.132993,18.507337,21.897599,18.443126,2.032870,...,1.571961,1.873212,1.130764,1.803368,1.822287,1.088816,2.605861,1.477076,1.977398,1.111402


In [ ]:
train_color_x = train_color.drop(columns=['Unnamed: 0','type', 'type_num'], axis=1)
train_color_y = train_color['type_num']
test_color_x = test_color.drop(columns=['Unnamed: 0'], axis=1)

In [8]:
test_x

,fiberID,psfMag_g,psfMag_i,fiberMag_g,fiberMag_i,petroMag_g,petroMag_i,modelMag_g,modelMag_i,fiber_u-g,...,psf_r-i,psf_i-z,petro_u-g,petro_g-r,petro_r-i,petro_i-z,model_u-g,model_g-r,model_r-i,model_i-z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,22.508963,18.517316,23.167848,18.835858,22.796239,18.584486,22.499435,18.499341,1.886041,...,2.463790,1.441237,-0.549543,1.600924,2.610829,1.430202,2.892099,1.487517,2.512577,1.407867
199992,386,21.937111,20.000512,22.186000,20.301204,21.837511,19.967204,21.853442,19.796757,0.312564,...,0.335258,0.473143,-0.107680,1.641382,0.228925,0.283532,0.621896,1.680273,0.376412,0.229385
199993,232,19.235669,17.808608,19.439533,17.909690,18.710223,17.158519,18.653338,17.120529,1.766013,...,0.495453,0.428495,2.012406,1.098371,0.453332,0.314533,1.925976,1.091230,0.441579,0.411782
199994,557,20.286261,20.162419,20.611498,20.479318,20.385262,20.206574,20.280943,20.206221,0.364634,...,0.034785,0.102587,-0.055993,0.256105,-0.077418,-0.005768,0.198937,0.130443,-0.055721,0.113311
199995,75,22.668237,19.284777,22.935289,19.624926,22.957496,19.299120,22.857290,19.237964,2.746572,...,1.954555,1.048838,-0.649198,1.672463,1.985913,0.991593,2.632069,1.665429,1.953898,0.957596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,19.542406,19.357957,19.827172,19.686861,19.549257,19.303361,19.536518,19.376445,0.266136,...,0.081328,0.000652,0.333472,0.091692,0.154203,-0.180028,0.255663,0.104056,0.056017,0.021469
209996,247,19.609379,19.391290,19.928523,19.763771,19.635609,19.460517,19.604800,19.401240,0.391391,...,-0.065341,0.132427,0.449845,0.253686,-0.078593,0.069652,0.461752,0.270688,-0.067127,0.241828
209997,941,20.276182,19.927208,20.551190,20.200872,20.315201,20.058801,20.255485,19.906014,0.703264,...,0.163567,0.160476,0.682819,0.078475,0.177925,0.070438,0.790016,0.113936,0.235535,-0.047918


In [22]:
d_train = lgbm.Dataset(train_x, label=train_y)
d_train
# d1_train = lgbm.Dataset(train_x1, label=train_y)

In [ ]:
d1_train = lgbm.Dataset(train_color_x, label=train_color_y)
d1_train

In [24]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':19,
    'metric': 'multi_logloss',
          'is_training_metric': True,
    'learning_rate': 0.005,
          'metric_freq': 3,
    'num_leaves': 123,
         'max_depth': 14,
'feature_fraction': 0.7,
'bagging_fraction': 0.7,
'bagging_freq': 15,
          'extra_trees' : True,
'verbose': 1}

# 'extra_trees' : True
# 0.8/0.7

In [25]:
%%time
lgb_cv = lgbm.cv(params, d_train, num_boost_round=2000, nfold=3, shuffle=True, stratified=True, verbose_eval=100, early_stopping_rounds=150)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
print(nround)

model = lgbm.train(params, d_train, num_boost_round=nround)


[100]	cv_agg's multi_logloss: 1.26639 + 0.000509696
[200]	cv_agg's multi_logloss: 0.883573 + 0.000798868
[300]	cv_agg's multi_logloss: 0.676535 + 0.000800942
[400]	cv_agg's multi_logloss: 0.554725 + 0.000787372
[500]	cv_agg's multi_logloss: 0.480454 + 0.000704962
[600]	cv_agg's multi_logloss: 0.43398 + 0.000662069
[700]	cv_agg's multi_logloss: 0.404265 + 0.000635085
[800]	cv_agg's multi_logloss: 0.384823 + 0.000709221
[900]	cv_agg's multi_logloss: 0.371872 + 0.000749921
[1000]	cv_agg's multi_logloss: 0.363097 + 0.000845049
[1100]	cv_agg's multi_logloss: 0.357055 + 0.000913999
[1200]	cv_agg's multi_logloss: 0.352964 + 0.000987887
[1300]	cv_agg's multi_logloss: 0.350176 + 0.00103399
[1400]	cv_agg's multi_logloss: 0.34828 + 0.00107779
[1500]	cv_agg's multi_logloss: 0.347032 + 0.00111217
[1600]	cv_agg's multi_logloss: 0.346269 + 0.00119463
[1700]	cv_agg's multi_logloss: 0.345939 + 0.00122156
[1800]	cv_agg's multi_logloss: 0.345921 + 0.00125996
[1900]	cv_agg's multi_logloss: 0.346133 + 0.00

In [ ]:
[1700]	cv_agg's multi_logloss: 0.345991 + 0.00276936
1734
CPU times: user 2h 58min, sys: 46.7 s, total: 2h 58min 46s
Wall time: 44min 45s
#        'num_leaves': 153,
#          'max_depth': 19,
    
    [100]	cv_agg's multi_logloss: 0.905295 + 0.00081828
[200]	cv_agg's multi_logloss: 0.58022 + 0.00110906
[300]	cv_agg's multi_logloss: 0.46017 + 0.00129014
[400]	cv_agg's multi_logloss: 0.410917 + 0.0014005
[500]	cv_agg's multi_logloss: 0.389026 + 0.00141282
[600]	cv_agg's multi_logloss: 0.378777 + 0.00159145
[700]	cv_agg's multi_logloss: 0.373884 + 0.00173854
[800]	cv_agg's multi_logloss: 0.371857 + 0.00191387
[900]	cv_agg's multi_logloss: 0.371468 + 0.0020348
890
CPU times: user 1h 54min 53s, sys: 37.1 s, total: 1h 55min 30s
Wall time: 29min 15s
    
#         'num_leaves': 132,
#          'max_depth': 15,
[100]	cv_agg's multi_logloss: 0.900856 + 0.00122878
[200]	cv_agg's multi_logloss: 0.574399 + 0.00148928
[300]	cv_agg's multi_logloss: 0.453668 + 0.00171016
[400]	cv_agg's multi_logloss: 0.403994 + 0.00202114
[500]	cv_agg's multi_logloss: 0.381623 + 0.00203409
[600]	cv_agg's multi_logloss: 0.370587 + 0.00214567
[700]	cv_agg's multi_logloss: 0.365241 + 0.00228484
[800]	cv_agg's multi_logloss: 0.362674 + 0.00247943
[900]	cv_agg's multi_logloss: 0.36154 + 0.00263081
[1000]	cv_agg's multi_logloss: 0.361279 + 0.00271221
990
CPU times: user 3h 44min 18s, sys: 10min 27s, total: 3h 54min 45s
Wall time: 1h 48min 6s
    

    
#       'num_leaves': 123,
#          'max_depth': 14, 
    
[100]	cv_agg's multi_logloss: 0.897186 + 0.000918592
[200]	cv_agg's multi_logloss: 0.570253 + 0.00143834
[300]	cv_agg's multi_logloss: 0.44968 + 0.0016027
[400]	cv_agg's multi_logloss: 0.400277 + 0.00173703
[500]	cv_agg's multi_logloss: 0.37833 + 0.00191453
[600]	cv_agg's multi_logloss: 0.367978 + 0.00209499
[700]	cv_agg's multi_logloss: 0.363202 + 0.00225808
[800]	cv_agg's multi_logloss: 0.361193 + 0.00240674
[900]	cv_agg's multi_logloss: 0.360812 + 0.00261106
884
CPU times: user 1h 48min 41s, sys: 56.4 s, total: 1h 49min 38s
Wall time: 28min 57s


#         'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.8,
# 'bagging_fraction': 0.8,
# 'bagging_freq': 15,
# 'verbose': 1
[100]	cv_agg's multi_logloss: 0.896205 + 0.00129673
[200]	cv_agg's multi_logloss: 0.569753 + 0.00187718
[300]	cv_agg's multi_logloss: 0.449015 + 0.00204485
[400]	cv_agg's multi_logloss: 0.399615 + 0.00221505
[500]	cv_agg's multi_logloss: 0.377465 + 0.00238236
[600]	cv_agg's multi_logloss: 0.367003 + 0.00253943
[700]	cv_agg's multi_logloss: 0.362078 + 0.00273581
[800]	cv_agg's multi_logloss: 0.359935 + 0.00290265
[900]	cv_agg's multi_logloss: 0.359392 + 0.00305269
[1000]	cv_agg's multi_logloss: 0.359822 + 0.00323563
901
CPU times: user 1h 39min 12s, sys: 36.3 s, total: 1h 39min 48s
Wall time: 25min 20s
    
    
#     'learning_rate': 0.005,
#           'metric_freq': 3,
#     'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.7,
# 'bagging_fraction': 0.6,
# 'bagging_freq': 17,
# 'verbose': 1}

[1000]	cv_agg's multi_logloss: 0.377104 + 0.00210881
[1100]	cv_agg's multi_logloss: 0.370757 + 0.0022219
[1200]	cv_agg's multi_logloss: 0.366383 + 0.00232922
[1300]	cv_agg's multi_logloss: 0.363296 + 0.00241026
[1400]	cv_agg's multi_logloss: 0.361157 + 0.0025126
[1500]	cv_agg's multi_logloss: 0.359777 + 0.00260291
[1600]	cv_agg's multi_logloss: 0.358883 + 0.00275088
[1700]	cv_agg's multi_logloss: 0.358335 + 0.00285778
[1800]	cv_agg's multi_logloss: 0.358143 + 0.00291452
[1900]	cv_agg's multi_logloss: 0.358192 + 0.00303881
1827
CPU times: user 2h 52min 33s, sys: 46.5 s, total: 2h 53min 19s
Wall time: 43min 36s

#         'learning_rate': 0.005,
#           'metric_freq': 3,
#     'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.8,
# 'bagging_fraction': 0.8,
# 'bagging_freq': 17,
# 'verbose': 1}
[1100]	cv_agg's multi_logloss: 0.370788 + 0.00251227
[1200]	cv_agg's multi_logloss: 0.36653 + 0.00260475
[1300]	cv_agg's multi_logloss: 0.363606 + 0.00268971
[1400]	cv_agg's multi_logloss: 0.361609 + 0.00279664
[1500]	cv_agg's multi_logloss: 0.360317 + 0.00288211
[1600]	cv_agg's multi_logloss: 0.359464 + 0.00299929
[1700]	cv_agg's multi_logloss: 0.35902 + 0.00308992
[1800]	cv_agg's multi_logloss: 0.358856 + 0.00313186
[1900]	cv_agg's multi_logloss: 0.358927 + 0.0031865
1800
CPU times: user 3h 10min 23s, sys: 1min 1s, total: 3h 11min 25s
Wall time: 48min 21s

    
#         'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.6,
# 'bagging_fraction': 0.5,
# 'bagging_freq': 17,
# 'verbose': 1}
[1000]	cv_agg's multi_logloss: 0.37807 + 0.00194611
[1100]	cv_agg's multi_logloss: 0.371489 + 0.00206374
[1200]	cv_agg's multi_logloss: 0.366889 + 0.00212652
[1300]	cv_agg's multi_logloss: 0.36368 + 0.00223426
[1400]	cv_agg's multi_logloss: 0.361383 + 0.00233367
[1500]	cv_agg's multi_logloss: 0.35989 + 0.00242336
[1600]	cv_agg's multi_logloss: 0.358872 + 0.00250527
[1700]	cv_agg's multi_logloss: 0.3583 + 0.00263682
[1800]	cv_agg's multi_logloss: 0.358049 + 0.00271176
[1900]	cv_agg's multi_logloss: 0.358078 + 0.0027945
1838
CPU times: user 2h 44min 43s, sys: 53.2 s, total: 2h 45min 36s
Wall time: 41min 49s



In [ ]:
## season2
#     'num_class':19,
#     'metric': 'multi_logloss',
#           'is_training_metric': True,
#     'learning_rate': 0.005,
#           'metric_freq': 3,
#     'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.8,
# 'bagging_fraction': 0.7,
# 'bagging_freq': 17,
# 'verbose': 1}
[100]	cv_agg's multi_logloss: 1.26508 + 0.000556789
[200]	cv_agg's multi_logloss: 0.882557 + 0.000778792
[300]	cv_agg's multi_logloss: 0.67555 + 0.00108148
[400]	cv_agg's multi_logloss: 0.554032 + 0.0013097
[500]	cv_agg's multi_logloss: 0.480033 + 0.00149794
[600]	cv_agg's multi_logloss: 0.433765 + 0.00168909
[700]	cv_agg's multi_logloss: 0.404186 + 0.00189655
[800]	cv_agg's multi_logloss: 0.384865 + 0.00199967
[900]	cv_agg's multi_logloss: 0.372029 + 0.0021227
[1000]	cv_agg's multi_logloss: 0.363317 + 0.00222809
[1100]	cv_agg's multi_logloss: 0.35738 + 0.00232255
[1200]	cv_agg's multi_logloss: 0.353336 + 0.00238839
[1300]	cv_agg's multi_logloss: 0.350531 + 0.00247724
[1400]	cv_agg's multi_logloss: 0.34874 + 0.00256688
[1500]	cv_agg's multi_logloss: 0.347564 + 0.00263807
[1600]	cv_agg's multi_logloss: 0.346906 + 0.00276307
[1700]	cv_agg's multi_logloss: 0.346628 + 0.00280742
[1800]	cv_agg's multi_logloss: 0.346647 + 0.00286113
1725
CPU times: user 3h 10min 23s, sys: 39.7 s, total: 3h 11min 3s
Wall time: 47min 49s

#     'learning_rate': 0.005,
#           'metric_freq': 3,
#     'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.8,
# 'bagging_fraction': 0.8,
# 'bagging_freq': 17,
#           'extra_trees' : True,
# 'verbose': 1}
    
    
    [100]	cv_agg's multi_logloss: 1.26458 + 0.000646887
[200]	cv_agg's multi_logloss: 0.882083 + 0.000776379
[300]	cv_agg's multi_logloss: 0.675148 + 0.000977762
[400]	cv_agg's multi_logloss: 0.55373 + 0.00118209
[500]	cv_agg's multi_logloss: 0.479739 + 0.00138397
[600]	cv_agg's multi_logloss: 0.433487 + 0.00157786
[700]	cv_agg's multi_logloss: 0.40394 + 0.0017723
[800]	cv_agg's multi_logloss: 0.384642 + 0.00196039
[900]	cv_agg's multi_logloss: 0.371859 + 0.00206765
[1000]	cv_agg's multi_logloss: 0.363219 + 0.00222903
[1100]	cv_agg's multi_logloss: 0.35729 + 0.00231427
[1200]	cv_agg's multi_logloss: 0.353193 + 0.00237062
[1300]	cv_agg's multi_logloss: 0.350474 + 0.00246236
[1400]	cv_agg's multi_logloss: 0.348658 + 0.00256188
[1500]	cv_agg's multi_logloss: 0.347509 + 0.00259153
[1600]	cv_agg's multi_logloss: 0.346841 + 0.00271289
[1700]	cv_agg's multi_logloss: 0.346531 + 0.00282786
[1800]	cv_agg's multi_logloss: 0.346542 + 0.00288455
1733
CPU times: user 3h 19min 43s, sys: 38.7 s, total: 3h 20min 22s
Wall time: 50min 9s

    
#       'learning_rate': 0.005,
#           'metric_freq': 3,
#     'num_leaves': 123,
#          'max_depth': 14,
# 'feature_fraction': 0.7,
# 'bagging_fraction': 0.7,
# 'bagging_freq': 17,
#           'extra_trees' : True,
# 'verbose': 1}    
    
    [100]	cv_agg's multi_logloss: 1.26619 + 0.000597669
[200]	cv_agg's multi_logloss: 0.883663 + 0.000778465
[300]	cv_agg's multi_logloss: 0.676339 + 0.00109144
[400]	cv_agg's multi_logloss: 0.554568 + 0.00127427
[500]	cv_agg's multi_logloss: 0.480325 + 0.00146713
[600]	cv_agg's multi_logloss: 0.433905 + 0.00162714
[700]	cv_agg's multi_logloss: 0.404216 + 0.00183761
[800]	cv_agg's multi_logloss: 0.384739 + 0.00197542
[900]	cv_agg's multi_logloss: 0.371809 + 0.00207111
[1000]	cv_agg's multi_logloss: 0.363051 + 0.00219526
[1100]	cv_agg's multi_logloss: 0.357057 + 0.00228445
[1200]	cv_agg's multi_logloss: 0.352924 + 0.00234627
[1300]	cv_agg's multi_logloss: 0.350094 + 0.00243865
[1400]	cv_agg's multi_logloss: 0.348218 + 0.0025357
[1500]	cv_agg's multi_logloss: 0.346998 + 0.00259985
[1600]	cv_agg's multi_logloss: 0.346291 + 0.00270795
[1700]	cv_agg's multi_logloss: 0.345991 + 0.00276936
[1800]	cv_agg's multi_logloss: 0.345945 + 0.00280523
1736
CPU times: user 3h 10min 34s, sys: 51.2 s, total: 3h 11min 25s
Wall time: 47min 58s

In [26]:
test_x

,fiberID,psfMag_g,psfMag_i,fiberMag_g,fiberMag_i,petroMag_g,petroMag_i,modelMag_g,modelMag_i,fiber_u-g,...,psf_g-r,psf_r-i,psf_i-z,petro_g-r,petro_r-i,petro_i-z,model_u-g,model_g-r,model_r-i,model_i-z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,22.508963,18.517316,23.167848,18.835858,22.796239,18.584486,22.499435,18.499341,1.886041,...,1.527857,2.463790,1.441237,1.600924,2.610829,1.430202,2.892099,1.487517,2.512577,1.407867
199992,386,21.937111,20.000512,22.186000,20.301204,21.837511,19.967204,21.853442,19.796757,0.312564,...,1.601341,0.335258,0.473143,1.641382,0.228925,0.283532,0.621896,1.680273,0.376412,0.229385
199993,232,19.235669,17.808608,19.439533,17.909690,18.710223,17.158519,18.653338,17.120529,1.766013,...,0.931608,0.495453,0.428495,1.098371,0.453332,0.314533,1.925976,1.091230,0.441579,0.411782
199994,557,20.286261,20.162419,20.611498,20.479318,20.385262,20.206574,20.280943,20.206221,0.364634,...,0.089057,0.034785,0.102587,0.256105,-0.077418,-0.005768,0.198937,0.130443,-0.055721,0.113311
199995,75,22.668237,19.284777,22.935289,19.624926,22.957496,19.299120,22.857290,19.237964,2.746572,...,1.428904,1.954555,1.048838,1.672463,1.985913,0.991593,2.632069,1.665429,1.953898,0.957596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,19.542406,19.357957,19.827172,19.686861,19.549257,19.303361,19.536518,19.376445,0.266136,...,0.103120,0.081328,0.000652,0.091692,0.154203,-0.180028,0.255663,0.104056,0.056017,0.021469
209996,247,19.609379,19.391290,19.928523,19.763771,19.635609,19.460517,19.604800,19.401240,0.391391,...,0.283430,-0.065341,0.132427,0.253686,-0.078593,0.069652,0.461752,0.270688,-0.067127,0.241828
209997,941,20.276182,19.927208,20.551190,20.200872,20.315201,20.058801,20.255485,19.906014,0.703264,...,0.185407,0.163567,0.160476,0.078475,0.177925,0.070438,0.790016,0.113936,0.235535,-0.047918


In [27]:
y_pred = model.predict(test_x) 

In [28]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/dacon_g/submission_LGBM_25(drop=pet_u).csv', index=True)

In [ ]:
[1]	cv_agg's multi_logloss: 2.24807 + 0.0002033
[2]	cv_agg's multi_logloss: 2.20302 + 0.000239095
[3]	cv_agg's multi_logloss: 2.1618 + 0.000248898
[4]	cv_agg's multi_logloss: 2.12361 + 0.000249803
[5]	cv_agg's multi_logloss: 2.08791 + 0.000263409
[6]	cv_agg's multi_logloss: 2.05409 + 0.000281898
[7]	cv_agg's multi_logloss: 2.02218 + 0.000258941
[8]	cv_agg's multi_logloss: 1.99161 + 0.000209379
[9]	cv_agg's multi_logloss: 1.96243 + 0.000218095
[10]	cv_agg's multi_logloss: 1.93455 + 0.000172731
9
CPU times: user 2min 38s, sys: 1.04 s, total: 2min 39s
Wall time: 40.1 s
    
# 221/17 - 0.015 (800)
[100]	cv_agg's multi_logloss: 0.697169 + 0.000536686
[200]	cv_agg's multi_logloss: 0.458259 + 0.00115001
[300]	cv_agg's multi_logloss: 0.396921 + 0.00145885
[400]	cv_agg's multi_logloss: 0.379244 + 0.00167057
[500]	cv_agg's multi_logloss: 0.37388 + 0.00167148
558
CPU times: user 1h 21min 13s, sys: 18.4 s, total: 1h 21min 32s
Wall time: 20min 27s
    
# Best
#         'num_leaves': 123,
#          'max_depth': 14,
[100]	cv_agg's multi_logloss: 0.910575 + 0.000879208
[200]	cv_agg's multi_logloss: 0.58546 + 0.00103805
[300]	cv_agg's multi_logloss: 0.464694 + 0.00119873
[400]	cv_agg's multi_logloss: 0.414358 + 0.00134551
[500]	cv_agg's multi_logloss: 0.391522 + 0.00130089
[600]	cv_agg's multi_logloss: 0.380271 + 0.00148962
[700]	cv_agg's multi_logloss: 0.374465 + 0.00168015
[800]	cv_agg's multi_logloss: 0.371536 + 0.00172819
[900]	cv_agg's multi_logloss: 0.370114 + 0.0017683
[1000]	cv_agg's multi_logloss: 0.369726 + 0.00191425
999
CPU times: user 1h 46min 55s, sys: 32.4 s, total: 1h 47min 28s
Wall time: 27min 7s